# TaxiSimulation examples

- typical import

In [3]:
using TaxiSimulation, RoutingNetworks, JLD

## Creating TaxiProblems

The `TaxiProblem` class represents problems we are trying to solve. Its most important features are:
- a `Network` object from the `RoutingNetworks` package, together with a `RoutingPaths` object from the same package. These types represent the static routing graph and directions that will be used by taxis. Another `RoutingPaths` object describes the costs. The hypothesis is that taxis will use shortest paths in time (and not in cost)
- a set of `Customers` (all customers to appear)
- a set of `Taxis` (all taxis on the map)

###  Synthetic problems

- We create here a typical small-sized routing problem on a synthetic city

In [9]:
# the network
n = urbanNetwork(8, distance=800.)

# the travel-times and paths. Here with road-types maximal speeds 
# (see RoutingNetworks for more details)
routing = roadTypeRouting(n)

#usually,the costs are proportional to the times (but not necessarily)
# here the cost is $5 per hour of driving
costs = RoutingPaths(n, routing.times*5./3600.)

# We create the taxi problem
# - customerTime = nb of seconds to pickup or dropoff a customer
# - waitingCost = nb of $ per second of a taxi waiting. (here $1/hour)
pb = TaxiProblem(n,routing,costs,customerTime= 10., waitingCost = 1./3600.)

# The problem is still incomplete, we need to add customer and taxis
# we first add random customers, see `taxiproblem/randomproblem.jl` for more info
# 1h of customer pickups, 0.35 customers per node per hour, fare = $80/hour, 
# customers can wait up to 5min after pickup time, and call 30min before
addRandomCustomers!(pb, 3600., 0.35, hourFare=80., custWait=5.0*60, custCall= 30.0*60)

# 20 uniformly distributed identical random taxis, all available at beginning of simulation
addRandomTaxis!(pb, 20)

# save this problem locally for latter use
save("smallurb.jld", "pb", pb)
pb

Taxi Problem
City with 192 locations and 640 roads
Simulation with 72 customers and 20 taxis for 60.00 minutes


# Timings Inference examples 

Typical import 

In [5]:
using TaxiSimulation, RoutingNetworks,JLD

# Network timings optimization
## Synthesizing network data
First, we create a synthetic network `n` (with tools from RoutingNetworks.jl). We associate it with a set of synthetic "true" timings `trueTimings`. Timings contains link times and times for each path.

Uncomment visualizations to see results

In [6]:
n = urbanNetwork(8)
# visualize(n)

# random speeds (btw 0 and 130kph)
trueRouting = randomTimings(n)

# another possibility: constant predefined speed for each road-type
trueTimings = roadTypeTimings(n)

# visualize(ShowTimes(n,trueTimings))

LoadError: LoadError: UndefVarError: randomTimings not defined
while loading In[6], in expression starting on line 5

Then, we create a synthetic set of trips based on this "true" times. This is stored as a `NetworkData` object, which contained all information for timings inference: network structure, trips and speed limits

In [ ]:
# All possible trips are given (for testing purposes)
pb = perfectVirtualData(trueTimings, 0.) 

# Generate data that tries to mimic real rides in a uniform city. 
pb = noisyVirtualData(trueTimings, 0.2, 2., timeStd = 20.)

## Iterative methods
The `IterativeState` object represents the state of an iterative time-estimation algorithm. It contains all the data, the current computed times, the set of paths that is used...

- `LimitedPath` is an implementation, that computes iteratively a set of paths for a subset of the trip data. A limit on the number of paths can be set.

The different methods available to compute new times are:
- `lp`: minimize MAPE (Mean absolute percentage error), uses Gurobi. `lpCo` and `lpCoNbhd` (Nbhd == Neighborhood) just add continuity constraints.
- `socp`: minimize MRE (Max ratio error), uses Mosek. `socpCo` and `Nbhd` (Nbhd == Neighborhood) just add continuity constraints.
- `mip`: implements the "minimum" constraints

In [ ]:
# first, initialize the iterative method with a set of times.
# Several options here
initialTimes = uniformTimes(n,50) #speed 50kph
initialTimes = pb.minTimes # maximal allowed speed
initialTimes = randomTimes(n);

# them, create the state
s = LimitedPaths(pb, initialTimes, pathsPerTrip = 3, maxTrip=1000)

# do a number of iterations of a desired algorithm
for i = 1:5
    doIteration!(s, method="lp", OutputFlag=0)
    println("Iteration $i, pathDiff=$(s.pathDiff)")
end
# solver args can also be given
# pathDiff measures how much the last iteration changed the path 
# (measure of convergence)

## Stats and visualization
To understand the evolution of iterative methods, we feed the results of each iteration to a NetworkStats object. We then use this object to query and visualize the data.

- First, we modify the loop to collect the information

In [ ]:
stats = VirtNetworkStats[] # this type of statistics requires that we have access to "true" times.

initialTimes = randomTimes(n);
s = LimitedPaths(pb, initialTimes, pathsPerTrip = 3, maxTrip=1000)

push!(stats, VirtNetworkStats("start", s, trueTimings))

# save stats at each iteration
for i = 1:5
    doIteration!(s, method="lp", OutputFlag=0)
    push!(stats, VirtNetworkStats("Iter $i", s, trueTimings))
    println("Iteration $i, pathDiff=$(s.pathDiff)")
end

- Now, we can either print stats about a particular iteration, or print or plot the evolution of a particular statistics accross a sequence of iterations

In [ ]:
printStats(stats[end]) #stats of last iteration

In [ ]:
printStats(stats, "allPathsLogError")

In [ ]:
plotStats(stats, "allPathsLogError")

- We can also inspect the evolution of the link times through an interactive visualization:
    - `SPACE` and `B` to respectively move to the next or previous iteration.
    - `ARROWS` to move around
    - `Z` and `X` to move in/out
    - `A` and `S` to increase/decrease the drawing size
    - `D` to show/hide the nodes of the network
    - `Q` and `ESC` to quit

In [ ]:
visualize(ShowTimes(n, stats, trueTimings))

# Working with real data
We present here a study using Manhattan's taxi data
## Setting up the network
The first step is to create the desired routing network. This can be done with the package RoutingNetworks, from a polygon. In our case we will use the predefined MANHATTAN polygon.
- get data (can take some time)

In [ ]:
MAN = getPolygon("Manhattan"); #Manhattan is hardcoded
manhattanNetwork = queryOsmPolygon(MAN)

- format the network to our need

In [ ]:
manhattanNetwork = roadTypeSubset(manhattanNetwork,1:6) # only keep main driving roads
manhattanNetwork = stronglyConnected(manhattanNetwork,1) # only keep the main connected component (hoping it's the one of node 1)
manhattanNetwork = intersections(manhattanNetwork) # simplify enormously the network, so that each node correspond to an intersection

- save the network to a julia JLD file for latter use

In [ ]:
saveTemplate(manhattanNetwork, "Manhattan")

- to load the network, just do:

In [ ]:
manhattanNetwork = loadTemplate("Manhattan")
# visualize(n)

## Formatting / loading the data
- Timings Inference have function to automatically load of NTC data. The CSVs can be found on their [website](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). 


In [ ]:
# Download data from website (will take some time, size = 1.8gb)
const DATA_URL_04_2016 = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-04.csv"
download(DATA_URL_04_2016, "taxidata042016.csv")

- We transform this data into a set of `GeoTrip`, which represents one trip in continuous coordinates. Note that a `GeoTrip` also contains the DateTime of the trip. To load from other datasets, one needs to write a converting function.

In [ ]:
# loads the data as a set of GeoTrip (long computation!)
trips = fromNYCTaxiCSV("taxidata042016.csv");

- cleaning the data and subsetting it

In [ ]:
trips = removeOutliers(trips) # remove trips that are likely to be bad data-points
trips = inPolygon(trips, getPolygon("Manhattan")) # origin and destination with specified polygon
trips = onlyWeekdays(trips) # another filter
trips = inTimeWindow(trips, "9:00", "11:00"); # restrict to a specific time of the day
println(length(trips), " trips")

- saving for latter use and deleting temporary files

In [ ]:
save("taxitrips.jld", "trips", trips)
rm("taxidata042016.csv")

In [ ]:
trips = load("taxitrips.jld", "trips");

## Timing Prediction
- First, we separate the data in a training and a testing set. The `DataSplit` type is a helper.

In [ ]:
TRAINING_FRACTION = 0.6
dataSplit = RandomSplit(trips, TRAINING_FRACTION)

- From there, we can directly construct a timings estimator (`GeoTimings` object) on the training set, and use it to compute times for the testing set.

In [ ]:
K = 15
# K nearest-neighbor estimator
knnEstimator = KnnTimings(dataSplit, K)

- In order to evaluate how well this estimator performs on the data, we can use a GeoStats object

In [ ]:
stats = RealGeoStats("15-NN stats", knnEstimator, dataSplit)

In [ ]:
printStats(stats)

## Projecting and using Network estimators
- In order to use `NetworkTimings` solver with continuous data, we use `NetworkProjector` types as a bridge between the two: 
    - to create `NetworkData` from `GeoData`
    - to use `NetworkTimings` to get `GeoTimings`

First, create a projector:

In [ ]:
proj = AvgRadius(manhattanNetwork, 200, trips) # project on all nodes within a 4-D sphere
proj = NearestRoad(manhattanNetwork, trips) # project onto the nearest road
proj = NearestNode(manhattanNetwork, trips) # project onto the nearest node in the network

Then, create a NetworkData from the continuous data using the projector (notice that the number of trip may vary when switching to the discrete network representation, some of them are combined):


In [ ]:
pb = NetworkData(proj, # the projector
                 trainSet(dataSplit), # the data we want to use
                 maxSpeedTimes(manhattanNetwork)) # speed limits
# we will need this one latter.
pbTest = NetworkData(proj, testSet(dataSplit), maxSpeedTimes(manhattanNetwork))
pb

Now we can just use our network-based timings estimation methods. Notice the use of `TimingsFromNetwork` to access the continuous-estimator at each iteration and used to compute the continuous stats.. We use the `RealNetworkStats` to get the discrete stats.

In [ ]:
initialTimes = randomTimes(manhattanNetwork);
s = LimitedPaths(pb, initialTimes, pathsPerTrip = 3, maxTrip=5000)

# Network time inference statistics
networkStats = RealNetworkStats[]
push!(networkStats, RealNetworkStats("start", s, pbTest))

# Continuous time inference statistics
geoStats = RealGeoStats[]
geoEstimator = TimingsFromNetwork(NetworkTimings(s), proj) #use projector to turn network estimator into a geo estimator
push!(geoStats, RealGeoStats("start", geoEstimator, dataSplit))

# save stats at each iteration
for i = 1:5
    # Infer new network timings
    doIteration!(s, method="lp", OutputFlag=0)
    
    #compute stats...
    push!(networkStats, RealNetworkStats("Iter $i", s, pbTest))
    geoEstimator = TimingsFromNetwork(NetworkTimings(s), proj)
    push!(geoStats, RealGeoStats("Iter $i", geoEstimator, dataSplit))

    println("Iteration $i, pathDiff=$(s.pathDiff)")
end

In [ ]:
printStats(networkStats, "testNetworkTripsLogError")
println()
printStats(geoStats, "testTripsLogError")

As before, we can show the evolution as a interactive visualization:

In [ ]:
visualize(ShowTimes(manhattanNetwork, networkStats))